<a href="https://colab.research.google.com/github/JuanpaQC/Beta01-Project/blob/main/Copy_Proyecto_taller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico de Costa Rica
### Reconocimiento de Rostros
Author: [Johan Rodríguez](https://github.com/Johanx22x)

Note: When using this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [85]:
!pip install face_recognition         # Install face_recognition package using pip
import face_recognition
import cv2                            # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt  # pyplot

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

from zmq.sugar.socket import T
from traitlets.config import Container
import pickle
import json

# Utils

## Automatic ID

In [ ]:
import uuid
import base64

def automatic_id():
  # Genera un UUID
  uuid_value = uuid.uuid4()

  # Codifica el UUID en Base64
  uuid_bytes = uuid_value.bytes
  base64_encoded = base64.b64encode(uuid_bytes).decode('utf-8')

  # Trunca el resultado para obtener un ID corto
  id_corto = base64_encoded[:8]  # Personaliza la longitud como desees

  return id_corto

## In-memory data

In [ ]:
registered_faces = {}

## face recognition

In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

## Real time streaming

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

## Register Teachers

In [ ]:
# Declare the global LoL_PersonalInfo
LoD_Teacher = [
    {
        'LoD_Teacher': {},
    }
]

# Save LoL_PersonalInfo in a binary file
with open('LoD_Teacher.bin', 'wb') as file:
    pickle.dump(LoD_Teacher, file)

# Verify that the data has been saved
print("LoD_Teacher saved successfully in 'LoD_Teacher.bin'.")

In [ ]:
def add_teacher():

# Global variable for LoL_PersonalInfo
  LoD_Teacher = [
      {
          'LoD_Teacher': {},
      }
  ]

  def load_data():
      try:
          with open('LoD_Teacher.bin', 'rb') as file:
              data = pickle.load(file)
          if isinstance(data, list) and len(data) == 1 and isinstance(data[0], dict) and 'LoD_Teacher' in data[0]:
              return data
          else:
              print("Invalid data format in 'LoD_Teacher.bin'. Using default data.")
      except FileNotFoundError:
          print("'LoD_Teacher.bin' not found. Using default data.")
      return LoD_Teacher

  def save_data(data):
      with open('LoD_Teacher.bin', 'wb') as file:
          pickle.dump(data, file)

  # Create a text input widget for the name
  name_input = widgets.Text(
      placeholder='Enter the name',
      description='Name:',
      disabled=False
  )

  id_input = widgets.Text(
      placeholder='Enter the ID:',
      description='ID:',
      disabled=False
  )

  date_of_birth_input = widgets.DatePicker(
    description='Date of birth:',
    disabled=False
  )

  nationallity_input = widgets.Text(
      placeholder='Enter the Nationallity:',
      description='Nationallity:',
      disabled=False
  )

  phone_input = widgets.Text(
      placeholder='Enter the Phone Number:',
      description='Number:',
      disabled=False
  )

  email_input = widgets.Text(
      placeholder='Enter the Email:',
      description='Email:',
      disabled=False
  )

  address_input = widgets.Text(
      placeholder='Enter the Teacher Address:',
      description='Address:',
      disabled=False
  )

  speciality_input = widgets.Text(
      placeholder='Enter the Teacher Speciality:',
      description='Speciality:',
      disabled=False
  )

  # Create a file upload widget
  upload_button = widgets.FileUpload(
      accept='image/*',
      multiple=False,
      description='Upload Image'
  )

  or_text = widgets.HTML(
      value=" or "
  )

  # Create a button to trigger face registration
  register_button = widgets.Button(description='Register Face')
  register_button.style.button_color = 'green'

  exit_button = widgets.Button(description="Back to Menu")
  exit_button.style.button_color = 'darkred'

  # Create a button to trigger image capture
  capture_button = widgets.Button(description='Capture Image')


  # Create an output widget to display the uploaded or captured face image
  output = widgets.Output()

  # Initialize variables to store the uploaded image and name
  uploaded_image_data = None
  registered_name = None

  # Define a function to handle image upload
  def handle_upload(change):
      global uploaded_image_data
      uploaded_filename = list(upload_button.value.keys())[0]
      uploaded_image_data = upload_button.value[uploaded_filename]['content']

      with output:
          clear_output()
          display(widgets.Image(value=uploaded_image_data))

      # Assuming uploaded_image_data is binary image data in bytes
      image_array = np.frombuffer(uploaded_image_data, np.uint8)
      image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

      # Convert the image to RGB format
      uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  def handle_capture(button):
    global uploaded_image_data
    # start streaming video from webcam
    video_stream()
    js_reply = video_frame("Capturing...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
      with output:
        clear_output()
        print("Error capturing image!")

    # convert JS response to OpenCV Image
    uploaded_image_data = js_to_image(js_reply["img"])

    with output:
      clear_output()
      # Encode the captured frame as PNG
      _, buffer = cv2.imencode('.png', uploaded_image_data)

      # Convert the encoded image to bytes
      image_bytes = buffer.tobytes()
      display(widgets.Image(value=image_bytes))

  # Define a function to handle face registration
  def handle_register(button):
      global registered_name
      global registered_id
      global registered_date_of_birth
      global registered_nationallity
      global registered_phone
      global registered_email
      global registered_address
      global registered_speciality

        # Wait for user input and store data
      id_teacher_input = id_input.value
      name_teacher_input = name_input.value
      date_of_birth_teacher_input = date_of_birth_input.value
      nationallity_teacher_input = nationallity_input.value
      phone_teacher_input  = phone_input.value
      email_teacher_input = email_input.value
      address_teacher_input  = address_input.value
      speciality_teacher_input = speciality_input.value

        # Create a dictionary for teacher information
      teacher_info = {
          'ID': str(id_input),
          'Name': str(name_input),
          'dateofbirth': str(date_of_birth_input),
          'nationallity': str(nationallity_input),
          'phone':str(phone_input),
          'email':str(email_input),
          'address':str(address_input),
          'speciality':str(speciality_input)
            # Add other fields here
      }

        # Append the teacher_info to the existing data
      LoD_Janitor[0]['LoD_Teacher'][str(id_input)] = teacher_info

        # Save the updated data directly to the file in append mode ('ab')
      with open('LoD_Teacher.bin', 'ab') as file:
          pickle.dump(LoD_Teacher, file)

      print("Data saved successfully!")


      if uploaded_image_data is not None:
          registered_name = name_input.value
          registres_id = id_input.value
          registres_date_of_birth = date_of_birth_input.value
          registres_nationallity = nationallity_input.value
          registres_phone = phone_input.value
          registres_email = email_input.value
          registres_address = address_input.value
          registres_speciality = speciality_input.value


          # Detect faces in the frame
          face_locations = face_recognition.face_locations(uploaded_image_data)

          # If there is no face, raise a popup window
          if len(face_locations) == 0:
              with output:
                clear_output()
                print("No face detected in the image!")
                return

          # If there is more than one face, raise a popup window
          if len(face_locations) > 1:
              with output:
                clear_output()
                print("More than one face detected in the image!")
                return

          # Get the face encodings
          face_encodings = face_recognition.face_encodings(uploaded_image_data)

          # Check if the face is registered
          if check_if_face_registered(face_encodings[0]):
              with output:
                clear_output()
                print("Face already registered!")
                return

          with output:
              clear_output()
              print(f"Face registered for {registered_name}")
              registered_faces[registered_name] = face_encodings[0]

  upload_button.observe(handle_upload, names='value')
  capture_button.on_click(handle_capture)
  register_button.on_click(handle_register)


  def back_menu(b):
    clear_output()
    dataAdder()
  exit_button.on_click(back_menu)

  # Create a container for the widgets
  container = widgets.VBox([name_input, id_input, date_of_birth_input, nationallity_input, phone_input, email_input, address_input, speciality_input, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([register_button, exit_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Teacher Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

## Register Students

In [ ]:
# Declare the global LoL_PersonalInfo
LoD_Students = [
    {
        'LoD_Students': {},
    }
]

# Save LoL_PersonalInfo in a binary file
with open('LoD_Students.bin', 'wb') as file:
    pickle.dump(LoD_Students, file)

# Verify that the data has been saved
print("LoD_Students saved successfully in 'LoD_Students.bin'.")

In [73]:
def add_student():

# Global variable for LoL_PersonalInfo
  LoD_Students = [
      {
          'LoD_Students': {},
      }
  ]

  def load_data():
      try:
          with open('LoD_Students.bin', 'rb') as file:
              data = pickle.load(file)
          if isinstance(data, list) and len(data) == 1 and isinstance(data[0], dict) and 'LoD_Students' in data[0]:
              return data
          else:
              print("Invalid data format in 'LoD_Students.bin'. Using default data.")
      except FileNotFoundError:
          print("'LoD_Students.bin' not found. Using default data.")
      return LoD_Students

  def save_data(data):
      with open('LoD_Students.bin', 'wb') as file:
          pickle.dump(data, file)

  # Create a text input widget for the name
  name_input = widgets.Text(
      placeholder='Enter the name',
      description='Name:',
      disabled=False
  )

  id_input = widgets.Text(
      placeholder='Enter the ID:',
      description='ID:',
      disabled=False
  )

  date_of_birth_input = widgets.DatePicker(
    description='Date of birth:',
    disabled=False
  )

  nationallity_input = widgets.Text(
      placeholder='Enter the Nationallity:',
      description='Nationallity:',
      disabled=False
  )

  phone_input = widgets.Text(
      placeholder='Enter the Phone Number:',
      description='Number:',
      disabled=False
  )

  email_input = widgets.Text(
      placeholder='Enter the Email:',
      description='Email:',
      disabled=False
  )

  address_input = widgets.Text(
      placeholder='Enter the Student Address:',
      description='Address:',
      disabled=False
  )

  opciones = [
              "Software Engineering",
              "Industrial Production Engineering",
              "Engineering in Agronomy",
              "Business Administration",
              "Electronics Engineering",
              "Computer Engineering",
              "Sustainable Rural Tourism Management",
              "Tourism Sustainability Management",
              ]
  career_dropdown = widgets.Dropdown(
    options=opciones,
    value=opciones[0],  # Valor inicial
    description='Career:'
  )

  # Create a file upload widget
  upload_button = widgets.FileUpload(
      accept='image/*',
      multiple=False,
      description='Upload Image'
  )

  or_text = widgets.HTML(
      value=" or "
  )

  # Create a button to trigger face registration
  register_button = widgets.Button(description='Register Face')
  register_button.style.button_color = 'green'

  exit_button = widgets.Button(description="Back to Menu")
  exit_button.style.button_color = 'darkred'

  # Create a button to trigger image capture
  capture_button = widgets.Button(description='Capture Image')

  # Create an output widget to display the uploaded or captured face image
  output = widgets.Output()

  # Initialize variables to store the uploaded image and name
  uploaded_image_data = None
  registered_name = None

  # Define a function to handle image upload
  def handle_upload(change):
      global uploaded_image_data
      uploaded_filename = list(upload_button.value.keys())[0]
      uploaded_image_data = upload_button.value[uploaded_filename]['content']

      with output:
          clear_output()
          display(widgets.Image(value=uploaded_image_data))

      # Assuming uploaded_image_data is binary image data in bytes
      image_array = np.frombuffer(uploaded_image_data, np.uint8)
      image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

      # Convert the image to RGB format
      uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  def handle_capture(button):
    global uploaded_image_data
    # start streaming video from webcam
    video_stream()
    js_reply = video_frame("Capturing...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
      with output:
        clear_output()
        print("Error capturing image!")

    # convert JS response to OpenCV Image
    uploaded_image_data = js_to_image(js_reply["img"])

    with output:
      clear_output()
      # Encode the captured frame as PNG
      _, buffer = cv2.imencode('.png', uploaded_image_data)

      # Convert the encoded image to bytes
      image_bytes = buffer.tobytes()
      display(widgets.Image(value=image_bytes))

  # Define a function to handle face registration
  def handle_register(button):
      global registered_name
      global registered_id
      global registered_date_of_birth
      global registered_nationallity
      global registered_phone
      global registered_email
      global registered_address
      global registered_career

        # Wait for user input and store data
      id_student_input = id_input.value
      name_student_input = name_input.value
      date_of_birth_student_input = date_of_birth_input.value
      nationallity_student_input = nationallity_input.value
      phone_student_input  = phone_input.value
      email_student_input = email_input.value
      address_student_input  = address_input.value
      shift_student_input = career_dropdown.value

        # Create a dictionary for teacher information
      student_info = {
          'ID': str(id_input),
          'Name': str(name_input),
          'dateofbirth': str(date_of_birth_input),
          'nationallity': str(nationallity_input),
          'phone':str(phone_input),
          'email':str(email_input),
          'address':str(address_input),
          'career':str(career_dropdown)
            # Add other fields here
      }

        # Append the teacher_info to the existing data
      LoD_Janitor[0]['LoD_Student'][str(id_input)] = student_info

        # Save the updated data directly to the file in append mode ('ab')
      with open('LoD_Student.bin', 'ab') as file:
          pickle.dump(LoD_Students, file)

      print("Data saved successfully!")

      if uploaded_image_data is not None:
          registered_name = name_input.value
          registres_id = id_input.value
          registres_date_of_birth = date_of_birth_input.value
          registres_nationallity = nationallity_input.value
          registres_phone = phone_input.value
          registres_email = email_input.value
          registres_address = address_input.value
          registres_carreer = career_dropdown.value


          # Detect faces in the frame
          face_locations = face_recognition.face_locations(uploaded_image_data)

          # If there is no face, raise a popup window
          if len(face_locations) == 0:
              with output:
                clear_output()
                print("No face detected in the image!")
                return

          # If there is more than one face, raise a popup window
          if len(face_locations) > 1:
              with output:
                clear_output()
                print("More than one face detected in the image!")
                return

          # Get the face encodings
          face_encodings = face_recognition.face_encodings(uploaded_image_data)

          # Check if the face is registered
          if check_if_face_registered(face_encodings[0]):
              with output:
                clear_output()
                print("Face already registered!")
                return

          with output:
              clear_output()
              print(f"Face registered for {registered_name}")
              registered_faces[registered_name] = face_encodings[0]

  upload_button.observe(handle_upload, names='value')
  capture_button.on_click(handle_capture)
  register_button.on_click(handle_register)

  def back_menu(b):
    clear_output()
    dataAdder()
  exit_button.on_click(back_menu)


  # Create a container for the widgets
  container = widgets.VBox([name_input, id_input, date_of_birth_input, nationallity_input, phone_input, email_input, address_input, career_dropdown, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([register_button, exit_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Student Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

## Register Janitor

In [ ]:
# Declare the global LoL_PersonalInfo
LoD_Janitor = [
    {
        'LoD_Janitor': {},
    }
]

# Save LoL_PersonalInfo in a binary file
with open('LoD_Janitor.bin', 'wb') as file:
    pickle.dump(LoD_Janitor, file)

# Verify that the data has been saved
print("LoD_Janitor saved successfully in 'LoD_Janitor.bin'.")

In [ ]:
def add_janitor():

  LoD_Janitor = [
      {
          'LoD_Janitor': {},
      }
  ]

  def load_data():
      try:
          with open('LoD_Janitor.bin', 'rb') as file:
              data = pickle.load(file)
          if isinstance(data, list) and len(data) == 1 and isinstance(data[0], dict) and 'LoD_Janitor' in data[0]:
              return data
          else:
              print("Invalid data format in 'LoD_Janitor.bin'. Using default data.")
      except FileNotFoundError:
          print("'LoD_Janitor.bin' not found. Using default data.")
      return LoD_Janitor

  def save_data(data):
      with open('LoD_Janitor.bin', 'wb') as file:
          pickle.dump(data, file)


  # Create a text input widget for the name
  name_input = widgets.Text(
      placeholder='Enter the name',
      description='Name:',
      disabled=False
  )

  id_input = widgets.Text(
      placeholder='Enter the ID',
      description='ID:',
      disabled=False
  )

  id_button = widgets.Button(
      description="Generate ID",
      value = " "
  )


  date_of_birth_input = widgets.DatePicker(
    description='Date of birth:',
    disabled=False
)

  nationallity_input = widgets.Text(
      placeholder='Enter the Nationallity:',
      description='Nationallity:',
      disabled=False
  )

  phone_input = widgets.Text(
      placeholder='Enter the Phone Number:',
      description='Phone:',
      disabled=False
  )

  email_input = widgets.Text(
      placeholder='Enter the Email:',
      description='Email:',
      disabled=False
  )

  address_input = widgets.Text(
      placeholder='Enter the Teacher Address:',
      description='Address:',
      disabled=False
  )


  opciones = ['Day', 'Night']
  shift_dropdown = widgets.Dropdown(
    options=opciones,
    value=opciones[0],  # Valor inicial
    description='Shift:'
)

  # Create a file upload widget
  upload_button = widgets.FileUpload(
      accept='image/*',
      multiple=False,
      description='Upload Image'
  )

  or_text = widgets.HTML(
      value=" or "
  )

# Función que se ejecutará cuando se haga clic en el botón
  def boton_clickeado(id_button):
    id_input.value = automatic_id()

  # Función que se ejecuta cuando se selecciona una fecha
  def fecha_seleccionada(change):
    fecha = change['new']
    #print(f'Fecha seleccionada: {fecha}')

# Función que se ejecuta cuando se cambia la selección
  def seleccion_cambiada(change):
    seleccion = change['new']
    #print(f'Se seleccionó: {seleccion}')

  # Registrar la función de cambio
  date_of_birth_input.observe(fecha_seleccionada, names='value')

  # Registrar la función de cambio
  shift_dropdown.observe(seleccion_cambiada, names='value')

  # Create a button to trigger face registration
  register_button = widgets.Button(description='Register Face')
  register_button.style.button_color = 'green'

  exit_button = widgets.Button(description="Back to Menu")
  exit_button.style.button_color = 'darkred'

  # Create a button to trigger image capture
  capture_button = widgets.Button(description='Capture Image')

  # Create an output widget to display the uploaded or captured face image
  output = widgets.Output()

  # Initialize variables to store the uploaded image and name
  uploaded_image_data = None
  registered_name = None

  # Define a function to handle image upload
  def handle_upload(change):
      global uploaded_image_data
      uploaded_filename = list(upload_button.value.keys())[0]
      uploaded_image_data = upload_button.value[uploaded_filename]['content']

      with output:
          clear_output()
          display(widgets.Image(value=uploaded_image_data))

      # Assuming uploaded_image_data is binary image data in bytes
      image_array = np.frombuffer(uploaded_image_data, np.uint8)
      image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

      # Convert the image to RGB format
      uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  def handle_capture(button):
    global uploaded_image_data
    # start streaming video from webcam
    video_stream()
    js_reply = video_frame("Capturing...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
      with output:
        clear_output()
        print("Error capturing image!")

    # convert JS response to OpenCV Image
    uploaded_image_data = js_to_image(js_reply["img"])

    with output:
      clear_output()
      # Encode the captured frame as PNG
      _, buffer = cv2.imencode('.png', uploaded_image_data)

      # Convert the encoded image to bytes
      image_bytes = buffer.tobytes()
      display(widgets.Image(value=image_bytes))

  # Define a function to handle face registration
  def handle_register(button):
      global registered_name
      global registered_id
      global registered_date_of_birth
      global registered_nationallity
      global registered_phone
      global registered_email
      global registered_address
      global registered_shift

        # Wait for user input and store data
      id_janitor_input = id_input.value
      name_janitor_input = name_input.value
      date_of_birth_janitor_input = date_of_birth_input.value
      nationallity_janitor_input = nationallity_input.value
      phone_janitor_input  = phone_input.value
      email_janitor_input = email_input.value
      address_janitor_input  = address_input.value
      shift_janitor_input = shift_dropdown.value

        # Create a dictionary for teacher information
      janitor_info = {
          'ID': str(id_input),
          'Name': str(name_input),
          'dateofbirth': str(date_of_birth_input),
          'nationallity': str(nationallity_input),
          'phone':str(phone_input),
          'email':str(email_input),
          'address':str(address_input),
          'shift':str(shift_dropdown)
            # Add other fields here
      }

        # Append the teacher_info to the existing data
      LoD_Janitor[0]['LoD_Janitor'][str(id_input)] = janitor_info

        # Save the updated data directly to the file in append mode ('ab')
      with open('LoD_Janitor.bin', 'ab') as file:
          pickle.dump(LoD_Janitor, file)

      print("Data saved successfully!")

      if uploaded_image_data is not None:
          registered_name = name_input.value
          registres_id = id_input.value
          registres_date_of_birth = date_of_birth_input.value
          registres_nationallity = nationallity_input.value
          registres_phone = phone_input.value
          registres_email = email_input.value
          registres_address = address_input.value
          registres_speciality = shift_dropdown.value


          # Detect faces in the frame
          face_locations = face_recognition.face_locations(uploaded_image_data)

          # If there is no face, raise a popup window
          if len(face_locations) == 0:
              with output:
                clear_output()
                print("No face detected in the image!")
                return

          # If there is more than one face, raise a popup window
          if len(face_locations) > 1:
              with output:
                clear_output()
                print("More than one face detected in the image!")
                return

          # Get the face encodings
          face_encodings = face_recognition.face_encodings(uploaded_image_data)

          # Check if the face is registered
          if check_if_face_registered(face_encodings[0]):
              with output:
                clear_output()
                print("Face already registered!")
                return

          with output:
              clear_output()
              print(f"Face registered for {registered_name}")
              registered_faces[registered_name] = face_encodings[0]

  upload_button.observe(handle_upload, names='value')
  capture_button.on_click(handle_capture)
  register_button.on_click(handle_register)
  id_button.on_click(boton_clickeado)

  def back_menu(b):
    clear_output()
    dataAdder()
  exit_button.on_click(back_menu)


  # Create a container for the widgets
  container = widgets.VBox([name_input,id_input, date_of_birth_input, nationallity_input, phone_input, email_input, address_input, shift_dropdown, widgets.HBox([upload_button, or_text, capture_button]), widgets.HBox([register_button, exit_button]), output])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Janitor Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)

# Recognize Faces

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # Get the face locations from the frame
  face_locations = face_recognition.face_locations(frame)

  # For each face in the frame
  for (top, right, bottom, left) in face_locations:
      # Draw a rectangle around the face
      cv2.rectangle(
          frame,
          (left, top),
          (right, bottom),
          (0, 0, 255),
          2,
      )
      # Get the face encoding of the face
      face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

      # Compare the face encoding to the faces in the database
      matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

      # If there is a match
      if True in matches:
          # Get the index of the match
          match_index = matches.index(True)

          # Get the name of the match
          match_name = list(registered_faces.keys())[match_index]

          # Draw the name of the match on the frame
          cv2.putText(
              frame,
              match_name,
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )
      else:
          # Draw unknown on the frame
          cv2.putText(
              frame,
              "Unknown",
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )

  # Show image
  plt.imshow(frame)
  plt.show()

# Add Course

In [ ]:
# Declare the global LoL_PersonalInfo
LoD_Course = [
    {
        'LoD_Course': {},
    }
]

# Save LoL_PersonalInfo in a binary file
with open('LoD_Course.bin', 'wb') as file:
    pickle.dump(LoD_Course, file)

# Verify that the data has been saved
print("LoD_Course saved successfully in 'LoD_Course.bin'.")

In [ ]:
def add_course():

# Global variable for LoL_PersonalInfo
  LoD_Course = [
      {
          'LoD_Course': {},
      }
  ]

  def load_data():
      try:
          with open('LoD_Course.bin', 'rb') as file:
              data = pickle.load(file)
          if isinstance(data, list) and len(data) == 1 and isinstance(data[0], dict) and 'LoD_Course' in data[0]:
              return data
          else:
              print("Invalid data format in 'LoD_Course.bin'. Using default data.")
      except FileNotFoundError:
          print("'LoL_LoD_Course.bin' not found. Using default data.")
      return LoD_Classroom

  def save_data(data):
      with open('LoD_Course.bin', 'wb') as file:
          pickle.dump(data, file)

  name_input = widgets.Text(
      placeholder='Enter the name',
      description='Name:',
      disabled=False
  )

  id_input = widgets.Text(
      placeholder='Enter the ID',
      description='ID:',
      disabled=False
  )

  period_input = widgets.Text(
      placeholder='Enter the Period',
      description='Period:',
      disabled=False
  )


  opciones = [
              "Software Engineering",
              "Industrial Production Engineering",
              "Engineering in Agronomy",
              "Business Administration",
              "Electronics Engineering",
              "Computer Engineering",
              "Sustainable Rural Tourism Management",
              "Tourism Sustainability Management",
              ]
  career_dropdown = widgets.Dropdown(
    options=opciones,
    value=opciones[0],  # Valor inicial
    description='Career:'
  )

  #lista de estudiantes

  #lista de profesores

  #lista de aulas

  schedule_input = widgets.Text(
      placeholder='Enter the Schedule',
      description='Schedule:',
      disabled=False
  )

    # Crear un botón
  save_button = widgets.Button(
      description="Save"
  )
  save_button.style.button_color = 'green'
  exit_button = widgets.Button(description="Back to Menu")
  exit_button.style.button_color = 'darkred'

  lista_id = [ ]
  # Función que se ejecutará cuando se haga clic en el botón
  def boton_clickeado(boton):
        # Wait for user input and store data
        id_course_input = id_input.value
        name_course_inout = name_input.value
        period_course_input = period_input.value
        carrer_course_input = career_dropdown.value
        schedule_course_input  = schedule_input.value


        # Create a dictionary for teacher information
        course_info = {
            'ID': str(id_input),
            'Name': str(name_input),
            'Period': str(period_input),
            'Carrer': str(career_dropdown),
            'Schedule': str(schedule_input),
            # Add other fields here
        }

        # Append the teacher_info to the existing data
        LoD_Course[0]['LoD_Course'][str(id_input)] = course_info

        # Save the updated data directly to the file in append mode ('ab')
        with open('LoD_Course.bin', 'ab') as file:
            pickle.dump(LoD_Course, file)

        print("Data saved successfully!")

  # Asignar la función de clic al botón
  save_button.on_click(boton_clickeado)

  # Crear un área de salida para mostrar mensajes
  output = widgets.Output()

  def back_menu(b):
    clear_output()
    dataAdder()
  exit_button.on_click(back_menu)

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Course Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(id_input, name_input, period_input, career_dropdown, schedule_input, widgets.HBox([save_button,exit_button]))

# Add Classroom

In [ ]:
# Declare the global LoL_PersonalInfo
LoD_Classroom = [
    {
        'LoD_Classroom': {},
    }
]

# Save LoL_PersonalInfo in a binary file
with open('LoD_Classroom.bin', 'wb') as file:
    pickle.dump(LoD_Classroom, file)

# Verify that the data has been saved
print("LoD_Classroom saved successfully in 'LoD_Classroom.bin'.")

In [ ]:
def add_classroom():

# Global variable for LoL_PersonalInfo
  LoD_Classroom = [
      {
          'LoD_Classroom': {},
      }
  ]

  def load_data():
      try:
          with open('LoD_Classroom.bin', 'rb') as file:
              data = pickle.load(file)
          if isinstance(data, list) and len(data) == 1 and isinstance(data[0], dict) and 'LoD_Classroom' in data[0]:
              return data
          else:
              print("Invalid data format in 'LoD_Classroom.bin'. Using default data.")
      except FileNotFoundError:
          print("'LoL_LoD_Classroom.bin' not found. Using default data.")
      return LoD_Classroom

  def save_data(data):
      with open('LoD_Classroom.bin', 'wb') as file:
          pickle.dump(data, file)


  id_input = widgets.Text(
      placeholder='Enter the ID',
      description='ID:',
      disabled=False
  )

  name_input = widgets.Text(
      placeholder='Enter the Name',
      description='Name:',
      disabled=False
  )

    # Crear un botón
  save_button = widgets.Button(
      description="Save"
  )
  save_button.style.button_color = 'green'
  exit_button = widgets.Button(description="Back to Menu")
  exit_button.style.button_color = 'darkred'

  #lista de miselaneos encargados
  #lista de horarios


  lista_id = [ ]
  # Función que se ejecutará cuando se haga clic en el botón
  def boton_clickeado(boton):
        # Wait for user input and store data
        id_class_input = id_input.value
        name_class_inout = name_input.value
        # Create a dictionary for teacher information
        classroom_info = {
            'ID': str(id_input),
            'Name': str(name_input),
            # Add other fields here
        }

        # Append the teacher_info to the existing data
        LoD_Classroom[0]['LoD_Classroom'][str(id_input)] = classroom_info

        # Save the updated data directly to the file in append mode ('ab')
        with open('LoL_PersonalInfo.bin', 'ab') as file:
            pickle.dump(LoD_Classroom, file)

        print("Data saved successfully!")

  # Asignar la función de clic al botón
  save_button.on_click(boton_clickeado)

  # Crear un área de salida para mostrar mensajes
  output = widgets.Output()

  def back_menu(b):
    clear_output()
    dataAdder()
  exit_button.on_click(back_menu)

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Add Classroom Details</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(id_input, name_input, widgets.HBox([save_button, exit_button]))

# Data Adder *LISTO*

In [ ]:
def dataAdder():
  # Crear un botón
  add_teacher_button = widgets.Button(description="Add Teacher")
  add_student_button = widgets.Button(description="Add Student")
  add_janitor_button = widgets.Button(description="Add Janitor")
  add_classroom_button = widgets.Button(description="Add Classroom")
  add_course_button = widgets.Button(description = "Add Course")
  back_menu_button = widgets.Button(description = "Back Menu")
  back_menu_button.style.button_color = 'darkred'

  def teacher_button(b):
    clear_output()
    add_teacher()
  def student_button(b):
    clear_output()
    add_student()
  def janitor_button(b):
    clear_output()
    add_janitor()
  def classroom_button(b):
    clear_output()
    add_classroom()
  def course_button(b):
    clear_output()
    add_course()
  def menu_button(b):
    clear_output()
    main()


  # Asignar la función de clic al botón
  add_teacher_button.on_click(teacher_button)
  add_student_button.on_click(student_button)
  add_janitor_button.on_click(janitor_button)
  add_classroom_button.on_click(classroom_button)
  add_course_button.on_click(course_button)
  back_menu_button.on_click(menu_button)

  # Mostrar el botón y el área de salida
  container = widgets.HBox([add_teacher_button, add_student_button, add_janitor_button, add_classroom_button, add_course_button, back_menu_button])

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Data Adder Menu</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)
dataAdder()

# Data Reader

In [61]:
def dataReader():
  # Crear un botón
  add_teacher_button = widgets.Button(description="Show Teachers")
  add_student_button = widgets.Button(description="Show Students")
  add_janitor_button = widgets.Button(description="Show Janitors")
  add_classroom_button = widgets.Button(description="Show Classrooms")
  add_course_button = widgets.Button(description = "show Courses")
  back_menu_button = widgets.Button(description = "Back Menu")
  back_menu_button.style.button_color = 'darkred'

  def teacher_button(b):
    clear_output()
    add_teacher()
  def student_button(b):
    clear_output()
    add_student()
  def janitor_button(b):
    clear_output()
    add_janitor()
  def classroom_button(b):
    clear_output()
    add_classroom()
  def course_button(b):
    clear_output()
    add_course()
  def menu_button(b):
    clear_output()
    main()


  # Asignar la función de clic al botón
  add_teacher_button.on_click(teacher_button)
  add_student_button.on_click(student_button)
  add_janitor_button.on_click(janitor_button)
  add_classroom_button.on_click(classroom_button)
  add_course_button.on_click(course_button)
  back_menu_button.on_click(menu_button)

  # Mostrar el botón y el área de salida
  container = widgets.HBox([add_teacher_button, add_student_button, add_janitor_button, add_classroom_button, add_course_button, back_menu_button])


  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Data Reader Menu</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)
  display(container)


# Menu Principal


In [ ]:
def main():
  # Crear un botón
  add_button = widgets.Button(description="Add Information")
  ask_button = widgets.Button(description="Ask Information")
  exit_button = widgets.Button(description="EXIT")
  exit_button.style.button_color = 'darkred'

    # Función que se ejecutará cuando se haga clic en el botón
  def add_information(b):
    clear_output()
    dataAdder()
  def ask_information(b):
    clear_output()
    dataReader()
  def exit(b):
    clear_output()
    display(finish_label)


  # Asignar la función de clic al botón
  add_button.on_click(add_information)
  ask_button.on_click(ask_information)
  exit_button.on_click(exit)

  html_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">Welcome To the System</h1>'
  )

  finish_label = widgets.HTML(
      value='<h1 style="color: white; font-size: 24px;">System Finished</h1>'
  )

  # Mostrar el widget en la celda de Jupyter Notebook
  display(html_label)

  # Mostrar el botón y el área de salida
  container = widgets.HBox([add_button, ask_button, exit_button])
  display(container)

main()